In [ ]:
import os

from torch.utils.data import DataLoader
import torch

from model import NARMWrapper
from torch_dataset import RecsysDataset, RecsysCollateFunction

from pytorch_lightning.loggers import MLFlowLogger
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks import (
    EarlyStopping,
    LearningRateMonitor,
    ModelCheckpoint,
    ModelSummary,
    StochasticWeightAveraging
)


In [ ]:
# Initialize MLFlow Logger

seed_everything(42) # Reproducibility purpose
experiment_name = "init_track_embedding"
run_name = f"{experiment_name}_e32_l4"
tracking_uri = f"file:experiments/{experiment_name}"

mlflow_logger = MLFlowLogger(
    experiment_name=experiment_name,
    run_name=run_name,
    tracking_uri=tracking_uri
)

# Hyperparameters
hparams = {
    "lr": 3e-3,
    "embedding_dim": 32,
    "hidden_size": 32,
    "n_gru_layers": 4,
    "n_items": 345712,
    "padding_idx": 345712,
    "embedding_matrix_path": None,#"data/vanilla_embeddings.npy",
    "k": 10,
    "batch_size": 128,
    "epochs": 20,
    "patience": 3,
}
mlflow_logger.log_hyperparams(hparams)

In [ ]:
# Data source 
root = "./data"
session_data = "split_session"

# Initialize dataloader
train_ds = RecsysDataset(data_path=os.path.join(root, session_data, "train.csv"))
val_ds   = RecsysDataset(data_path=os.path.join(root, session_data, "val.csv"))
test_ds  = RecsysDataset(data_path=os.path.join(root, session_data, "test.csv"))

collate_fn = RecsysCollateFunction(padding_idx=hparams["padding_idx"])

batch_size = hparams["batch_size"]
train_ld = DataLoader(dataset=train_ds, batch_size=batch_size, shuffle=True, num_workers=4, persistent_workers=True, collate_fn=collate_fn)
val_ld   = DataLoader(dataset=val_ds, batch_size=batch_size, shuffle=False, num_workers=4, persistent_workers=True, collate_fn=collate_fn)
test_ld  = DataLoader(dataset=test_ds, batch_size=batch_size, shuffle=False, num_workers=4, collate_fn=collate_fn)

In [ ]:
# Initialize model 
narm = NARMWrapper(embedding_dim=hparams["embedding_dim"], 
                   hidden_size=hparams["hidden_size"], 
                   n_gru_layers=hparams["n_gru_layers"], 
                   n_items=hparams["n_items"], 
                   padding_idx=hparams["padding_idx"],
                   embedding_matrix_path=hparams["embedding_matrix_path"], 
                   lr=hparams["lr"],
                   k = hparams["k"])

In [ ]:
# Callbacks 
check_point_name = f"best-checkpoint_{experiment_name}"

training_callbacks = [
        EarlyStopping(monitor="val_loss", mode="min", patience=hparams['patience']),
        StochasticWeightAveraging(swa_lrs=1e-2),
        LearningRateMonitor(logging_interval="step"),
        ModelCheckpoint(
            monitor="val_loss",
            mode="min",
            save_top_k=1,
            dirpath="checkpoints/",
            filename=f"best-checkpoint_{run_name}"
        ),
        ModelSummary(-1)    
    ]

In [ ]:
# Training
torch.cuda.empty_cache()  
trainer = Trainer(
    logger=mlflow_logger,
    callbacks=training_callbacks,
    max_epochs=hparams["epochs"],
    log_every_n_steps=1,
)

trainer.fit(model=narm,
            train_dataloaders=train_ld,
            val_dataloaders= val_ld, 
            ckpt_path=None)

In [ ]:
# Testing 
trainer.test(model=narm, 
        dataloaders=test_ld, 
        ckpt_path=None)